In [1]:
# Modules to use
from transformers import AutoTokenizer, DataCollatorWithPadding

C:\Users\Chaitanya Belwal\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Prepare the dataset

In [ ]:

checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Logic to run the model

 '_' is allowed between number for easier readibility
    vocab_size,max_len,d_k, d_model, n_heads,n_layers,n_classes,dropout_prob
    vocab_size = 20,000
    max_len = 1024
    d_k = 16
    d_model = 64
    n_heads = 4
    n_layers = 2
    n_classes = 5
    dropout_prob = 0.1

In [ ]:
import Encoder

model = Encoder(20_000, 1024, 16, 64, 4, 2, 5, 0.1)